# UMPS homework 1 template

Let's use the following convention for numbering legs:
```
 1--A--3
    |
    2
```

In [1]:
using TensorOperations
using LinearMaps

In [2]:
"""
    rand_UMPS(d, D; keep_it_real=true)

Return a random three-valent tensor A, that defines a uniform MPS (UMPS).
The bond dimension of the physical leg should be d, and the bond dimension
of the two "virtual" legs (the horizontal ones) should be D.
keep_it_real is keyword argument, for whether the matrix should be real or
complex.

This means you can call
`rand_UMPS(2, 9)`
or
`rand_UMPS(2, 9; keep_it_real=true)`
and they both give a you real A, but you can also call
`rand_UMPS(2, 9; keep_it_real=false)`
to get a complex A.
"""
function rand_UMPS(d, D; keep_it_real=true)
    shp = (D, d, D)
    if keep_it_real
        A = randn(shp)
    else
        A_real = randn(shp)
        A_imag = randn(shp)
        A = complex.(A_real, A_imag) / sqrt(2)
    end
    return A
end

rand_UMPS

In [3]:
"""
    tm(A)

Return the transfer matrix of A:
 --A---
   |  
 --A*--
"""
function tm(A)
    @tensor T[i1,i2,j1,j2] := A[i1,p,j1]*conj(A)[i2,p,j2]
end

tm

In [4]:
function eig_and_trunc(T, nev; by=identity, rev=false)
    S, U = eig(T)
    perm = sortperm(S; by=by, rev=rev)
    S = S[perm]
    U = U[:, perm]
    S = S[1:nev]
    U = U[:, 1:nev]
    return S, U
end

"""
    tm_eigs(A, dirn, nev)

Return some of the eigenvalues and vectors of the transfer matrix of A.
dirn should be "L", "R" or "BOTH", and determines which eigenvectors to return.
nev is the number of eigenpairs to return (starting with the eigenvalues with
largest magnitude).
"""
function tm_eigs_dense(A, dirn, nev)
    T = tm(A)
    D = size(T, 1)
    T = reshape(T, (D^2, D^2))
    nev = min(nev, D^2)
    
    result = ()
    if dirn == "R" || dirn == "BOTH"
        SR, UR = eig_and_trunc(T, nev; by=abs, rev=true)
        UR = [reshape(UR[:,i], (D, D)) for i in 1:nev]
        result = tuple(result..., SR, UR)
    end
    if dirn == "L" || dirn == "BOTH"
        SL, UL = eig_and_trunc(T', nev; by=abs, rev=true)
        UL = [reshape(UL[:,i], (D, D)) for i in 1:nev]
        result = tuple(result..., SL, UL)
    end
    return result
end

tm_eigs_dense

In [5]:
"""
    normalize!(A)

Normalize the UMPS defined by A. This is done by dividing A by the square root of
the dominant (largest magnitude) eigenvalue of the MPS transfer matrix.
"""
function normalize!(A)
    S, U = tm_eigs_dense(A, "R", 1)
    S1 = S[1]
    A ./= sqrt(S1)
    return A
end

normalize!

In [7]:
let
    d = 2
    D = 10
    A = rand_UMPS(d, D; keep_it_real=false)

    T = tm(A)

    SR, UR, SL, UL = tm_eigs_dense(A, "BOTH", 1)
    @show SR[1]
    @show SL[1]
    normalize!(A)
    SR, UR, SL, UL = tm_eigs_dense(A, "BOTH", 1)
    @show SR[1]
    @show SL[1]
end

SR[1] = 19.714485953136677 - 2.373101715136272e-15im
SL[1] = 19.714485953136524 + 1.7763568394002505e-15im
SR[1] = 0.9999999999999942 + 3.154011220651623e-17im
SL[1] = 0.9999999999999918 + 5.551115123125783e-16im


0.9999999999999918 + 5.551115123125783e-16im

### Hurray!
Now let's get smart about diagonalizing that MPS transfer matrix.

In [8]:
"""
    tm_l(A, x)

Return y, where
/------   /------A--
|       = |      |  
\- y* -   \- x* -A*-
"""
function tm_l(A, x)
    @tensor y[i, j] := (x[a, b] * A[b, p, j]) * conj(A[a, p, i])
    return y
end


"""
    tm_r(A, x)

Return y, where
-- y -\   --A-- x -\
      | =   |      |
------/   --A*-----/
"""
function tm_r(A, x)
    @tensor y[i, j] := A[i, p, a] * (conj(A[j, p, b]) * x[a, b])
    return y
end


tm_r

In [9]:
function tm_eigs_sparse(A, dirn, nev)
    if dirn == "BOTH"
        SR, UR = tm_eigs_sparse(A, "R", nev)
        SL, UL = tm_eigs_sparse(A, "L", nev)
        return SR, UR, SL, UL
    else
        D = size(A, 1)
        x = zeros(eltype(A), (D, D))
        if dirn == "L"
            f = v -> vec(tm_l(A, copy!(x, v)))
        else
            f = v -> vec(tm_r(A, copy!(x, v)))
        end

        fmap = LinearMap{eltype(A)}(f, D^2)
        S, U, nconv, niter, nmult, resid = eigs(fmap, nev=nev, which=:LM, ritzvec=true)
        U = [reshape(U[:,i], (D, D)) for i in 1:size(U, 2)]

        return S, U
    end
end

tm_eigs_sparse (generic function with 1 method)

In [10]:
let
    d = 2
    D = 10
    A = rand_UMPS(d, D; keep_it_real=false)

    nev = 2
    @time SR_sparse, UR_sparse = tm_eigs_sparse(A, "R", nev)
    @time SR_dense, UR_dense = tm_eigs_dense(A, "R", nev)
    println("\nComparison of eigenvalues:")
    println(SR_dense)
    println(SR_sparse)
    println("\nComparison of eigenvectors:")
    println(UR_dense[1][1:6])
    println(UR_sparse[1][1:6])
    println("\nComparison of abs.(eigenvectors):")
    println(abs.(UR_dense[1])[1:6])
    println(abs.(UR_sparse[1])[1:6])
end

  3.034392 seconds (2.22 M allocations: 121.207 MiB, 1.44% gc time)
  0.034784 seconds (181 allocations: 886.016 KiB)

Comparison of eigenvalues:
Complex{Float64}[20.3559+0.0im, -16.2995+3.19744e-14im]
Complex{Float64}[20.3559+3.56519e-17im, -16.2995-4.88282e-15im]

Comparison of eigenvectors:
Complex{Float64}[0.221192-4.16334e-17im, 0.0351306-0.0429335im, 0.0903609-0.00378906im, -0.0791865+0.0180698im, 0.0167815+0.0783496im, -0.0117289-0.00890225im]
Complex{Float64}[-0.0484767-0.215815im, -0.049589-0.0248672im, -0.0235005-0.0873337im, 0.0349851+0.0733012im, 0.0727669-0.0335447im, -0.00611532+0.0133948im]

Comparison of abs.(eigenvectors):
[0.221192, 0.0554747, 0.0904403, 0.0812221, 0.0801266, 0.0147247]
[0.221192, 0.0554747, 0.0904403, 0.0812221, 0.0801266, 0.0147247]


In [11]:
function tm_eigs(A, dirn, nev; max_dense_D=10)
    D = size(A, 1)
    if D <= max_dense_D || nev >= D^2
        return tm_eigs_dense(A, dirn, nev)
    else
        return tm_eigs_sparse(A, dirn, nev)
    end
end

tm_eigs (generic function with 1 method)

In [46]:
"""
    normalize!(A)

Normalize the UMPS defined by A, and return the dominant left and right
eigenvectors l and r of its transfer matrix, normalized so that l'*r = 1.
"""
function normalize!(A)
    SR, UR, SL, UL = tm_eigs(A, "BOTH", 1)
    S1 = SR[1]
    A ./= sqrt(S1)
    
    l = UL[1]
    r = UR[1]  
    #We need this to be 1
    n = vec(l)'*vec(r)
    abs_n = abs(n)
    phase_n = abs_n/n
    sfac = 1.0/sqrt(abs_n)
    l .*= sfac/phase_n
    r .*= sfac
    return l, r
end

normalize!

In [47]:
let
    d = 2
    D = 50
    A = rand_UMPS(d, D; keep_it_real=false)

    T = tm(A)

    SR, UR, SL, UL = tm_eigs(A, "BOTH", 1)
    @show SR[1]
    @show SL[1]
    l, r = normalize!(A)
    SR, UR, SL, UL = tm_eigs(A, "BOTH", 1)
    @show SR[1]
    @show SL[1]
    
    @show l ≈ tm_l(A, l)
    @show r ≈ tm_r(A, r)
    @show vec(l)'*vec(r)
end

SR[1] = 98.90034275131886 + 0.0im
SL[1] = 98.90034275131926 + 1.4210854715202004e-14im
SR[1] = 0.9999999999999987 - 1.1102230246251565e-16im
SL[1] = 1.0000000000000013 + 1.1102230246251565e-16im
l ≈ tm_l(A, l) = true
r ≈ tm_r(A, r) = true
(vec(l))' * vec(r) = 1.0000000000000002 + 0.0im


1.0000000000000002 + 0.0im

## Homework 2 template

In [41]:
"""
    expect_local(A, O; l=nothing, r=nothing)

Return the expectation value of the one-site operator O for the UMPS state
defined by the tensor A.
"""
function expect_local(A, O; l=nothing, r=nothing)
    if l == nothing || r == nothing
        l, r = normalize!(A)
    end
    # ???
end

expect_local

In [42]:
"""
    correlator_twopoint(A, O1, O2, m; l=nothing, r=nothing)

Return the (connected) two-point correlator of operators O1 and O2 for the
state UMPS(A), when O1 and O2 are i sites apart, where i ranges from 1 to m. In
other words, return <O1_0 O2_i> - <O1> <O2>, for all i = 1,...,m, where the
expectation values are with respect to the state |UMPS(A)>.
"""
function correlator_twopoint(A, O1, O2, m; l=nothing, r=nothing)
    if l == nothing || r == nothing
        l, r = normalize!(A)
    end
    # ???
end

correlator_twopoint

In [43]:
"""
    tm_l_op(A, O, x)

Return y, where
/------   /------A--
|         |      |  
|       = |      O  
|         |      |  
\- y* -   \- x* -A*-
"""
function tm_l_op(A, O, x)
    # ???
end


"""
    tm_r_op(A, O, x)

Return y, where
-- y -\   --A-- x -\
      |     |      |
      | =   O      |
      |     |      |
------/   --A*-----/t
"""
function tm_r_op(A, O, x)
    # ???
end


tm_r_op

In [44]:
"""
    correlation_length(A)

Return the correlation length ξ of the UMPS defined by A. ξ = - 1/ln(|lambda[2]|),
where lambda[2] is the eigenvalue of the MPS transfer matrix with second largest
magnitude. (We assume here that UMPS(A) is normalized.)
"""
function correlation_length(A)
    # ???
end

correlation_length